# Jersey Number Pipeline — Colab Setup & Run

**Before running:** Runtime → Change runtime type → **GPU** (e.g. T4).

Then run the cells in order. Edit the **CONFIG** cell to match your Google Drive paths.

## 1. CONFIG — Edit these to match your Google Drive

Put on Drive:
- Dataset zip (e.g. `jersey-2023.zip`) in your project folder.
- A `weights` folder with subfolders: `models/`, `reid/`, `pose/` (with the required .pth/.ckpt files).

In [ ]:
# ============== EDIT THESE ==============
DRIVE_BASE = "/content/drive/MyDrive"
DRIVE_PROJECT_FOLDER = "jersey-number-pipeline"  # folder under My Drive with your zip and weights
DATASET_ZIP = "jersey-2023.zip"
WEIGHTS_FOLDER = "weights"
MOUNT_DRIVE = True  # set False if you already mounted in a previous run
# =======================================

## 2. Clone repo and SAM

In [ ]:
!git clone https://github.com/superbolt08/jersey-number-pipeline.git
%cd jersey-number-pipeline
!git clone https://github.com/davda54/sam.git sam2

## 3. Mount Google Drive

In [ ]:
if MOUNT_DRIVE:
    from google.colab import drive
    drive.mount("/content/drive")

## 4. Copy dataset from Drive and unzip

In [ ]:
import os

repo_root = "/content/jersey-number-pipeline"
drive_project = os.path.join(DRIVE_BASE, DRIVE_PROJECT_FOLDER)
drive_zip = os.path.join(drive_project, DATASET_ZIP)

os.makedirs(f"{repo_root}/data/SoccerNet", exist_ok=True)
!cp "{drive_zip}" "{repo_root}/"
!unzip -o -q "{repo_root}/{DATASET_ZIP}" -d "{repo_root}/data/SoccerNet"

# If zip had train/ and test/ at top level, put them under jersey-2023
data_sn = f"{repo_root}/data/SoccerNet"
if os.path.isdir(f"{data_sn}/train") and not os.path.isdir(f"{data_sn}/jersey-2023"):
    !mkdir -p "{data_sn}/jersey-2023"
    !mv "{data_sn}/train" "{data_sn}/jersey-2023/"
    !mv "{data_sn}/test" "{data_sn}/jersey-2023/"

!ls "{data_sn}"/

## 5. Copy model weights from Drive

In [ ]:
drive_weights = os.path.join(drive_project, WEIGHTS_FOLDER)

!mkdir -p "{repo_root}/models" "{repo_root}/reid/centroids-reid/models" "{repo_root}/pose/ViTPose/checkpoints"
!cp "{drive_weights}/models/"* "{repo_root}/models/" 2>/dev/null || true
!cp "{drive_weights}/reid/"* "{repo_root}/reid/centroids-reid/models/" 2>/dev/null || true
!cp "{drive_weights}/pose/"* "{repo_root}/pose/ViTPose/checkpoints/" 2>/dev/null || true
print("Weights copied.")

## 6. Install dependencies

In [ ]:
!pip install -q torch torchvision opencv-python Pillow numpy pandas scipy tqdm pytorch-lightning yacs

## 7. Run the pipeline

In [ ]:
%cd /content/jersey-number-pipeline
!python main.py SoccerNet test

## 8. (Optional) Download results or copy to Drive

In [ ]:
# Zip and download to your computer
# !cd /content/jersey-number-pipeline && zip -r out.zip out/
# from google.colab import files
# files.download('out.zip')

# Or copy outputs to Drive
!cp -r /content/jersey-number-pipeline/out "{DRIVE_BASE}/{DRIVE_PROJECT_FOLDER}/"
print("Outputs copied to Drive.")